In [34]:
import csv
import re

rows = []
table_started = False
expected_columns = None
short_text_index = None

with open("3. EKPO - Mar'24.txt", "r", encoding="utf-8", errors="replace") as infile:
    for line in infile:
        line_stripped = line.strip()
        # Skip lines that are empty or contain only dashes/spaces
        if not line_stripped or re.fullmatch(r'[-\s]+', line_stripped):
            continue

        # Check for the table header. We assume the header contains "Purch.Doc."
        if "Purch.Doc." in line_stripped:
            table_started = True
            # Split on pipe or tab if pipe is not present
            if "|" in line_stripped:
                header = [field.strip() for field in line_stripped.split('|')]
            else:
                header = [field.strip() for field in re.split(r'\t+', line_stripped)]
            # Remove leading empty element if exists
            if header and header[0] == "":
                header = header[1:]
            rows.append(header)
            expected_columns = len(header)
            # Determine the index for "Short Text" (if it exists)
            try:
                short_text_index = header.index("Short Text")
            except ValueError:
                short_text_index = None
            continue

        # If the table has started, process subsequent lines as table data
        if table_started:
            if "|" in line_stripped:
                row = [field.strip() for field in line_stripped.split('|')]
            else:
                row = [field.strip() for field in re.split(r'\t+', line_stripped)]
            # Remove leading empty element if exists
            if row and row[0] == "":
                row = row[1:]
            # If we have a header and the "Short Text" column, and there are extra fields,
            # combine extra fields into the "Short Text" column.
            if expected_columns and short_text_index is not None and len(row) > expected_columns:
                extra = len(row) - expected_columns
                # Combine the fields that belong to "Short Text"
                combined = "|".join(row[short_text_index:short_text_index + extra + 1])
                # Build the new row: before Short Text, then combined field, then the rest
                row = row[:short_text_index] + [combined] + row[short_text_index + extra + 1:]
            
            # Only add the row if it contains data
            if row and any(field for field in row):
                rows.append(row)

with open("EKPO_Mar24.csv", "w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerows(rows)

print("CSV conversion complete. 'output.csv' has been created.")


CSV conversion complete. 'output.csv' has been created.


### For Converting text to csv without Duplicate columns

In [33]:
# Same code as above, but it is for converting text to csv without duplicate columns
import csv
import re

rows = []
table_started = False
expected_columns = None
short_text_index = None

with open("1. EKPO - Jan'24.txt", "r", encoding="utf-8", errors="replace") as infile:
    for line in infile:
        line_stripped = line.strip()
        # Skip lines that are empty or contain only dashes/spaces
        if not line_stripped or re.fullmatch(r'[-\s]+', line_stripped):
            continue

        # Check for the table header. We assume the header contains "Purch.Doc."
        if "Purch.Doc." in line_stripped:
            table_started = True
            # Split on pipe or on tab if pipe is not present
            if "|" in line_stripped:
                header = [field.strip() for field in line_stripped.split('|')]
            else:
                header = [field.strip() for field in re.split(r'\t+', line_stripped)]
            # Remove leading empty element if exists
            if header and header[0] == "":
                header = header[1:]
            rows.append(header)
            expected_columns = len(header)
            # Determine the index for "Short Text" (if it exists)
            try:
                short_text_index = header.index("Short Text")
            except ValueError:
                short_text_index = None
            continue

        # If the table has started, process subsequent lines as table data
        if table_started:
            if "|" in line_stripped:
                row = [field.strip() for field in line_stripped.split('|')]
            else:
                row = [field.strip() for field in re.split(r'\t+', line_stripped)]
            # Remove leading empty element if exists
            if row and row[0] == "":
                row = row[1:]
            # If we have a header and the "Short Text" column, and there are extra fields,
            # combine extra fields into the "Short Text" column.
            if expected_columns and short_text_index is not None and len(row) > expected_columns:
                extra = len(row) - expected_columns
                # Combine the fields that belong to "Short Text"
                combined = "|".join(row[short_text_index:short_text_index + extra + 1])
                # Build the new row: before Short Text, then combined field, then the rest
                row = row[:short_text_index] + [combined] + row[short_text_index + extra + 1:]
            # Only add the row if it contains data
            if row and any(field for field in row):
                rows.append(row)

# --- Deduplicate Columns: Remove duplicate headers and corresponding values ---
if rows:
    header = rows[0]
    seen = {}
    duplicate_indices = []
    new_header = []
    for idx, col in enumerate(header):
        if col not in seen:
            seen[col] = idx
            new_header.append(col)
        else:
            duplicate_indices.append(idx)
    # Replace header row with the deduplicated header
    rows[0] = new_header
    # Remove duplicate columns from every subsequent row.
    # Sorting in descending order to avoid index shifting.
    duplicate_indices = sorted(duplicate_indices, reverse=True)
    for i in range(1, len(rows)):
        row = rows[i]
        for idx in duplicate_indices:
            if idx < len(row):
                row.pop(idx)

# with open("jandataagain.csv", "w", newline="", encoding="utf-8") as outfile:
#     writer = csv.writer(outfile)
#     writer.writerows(rows)

# print("CSV conversion complete. 'output.csv' has been created.")


In [ ]:
### Concate Different month data after conversion

In [2]:
import pandas as pd
import glob

files = ["EKPO_Jan24.csv", "EKPO_Feb24.csv", "EKPO_Mar24.csv"]

df_list = [pd.read_csv(file) for file in files]

final_df = pd.concat(df_list, ignore_index=True)

final_df.to_csv("final_concatenated_output.csv", index=False)

print("All monthly data has been successfully concatenated into 'final_concatenated_output.csv'.")


C:\Users\HIMANSHU SURYAVANSHI\AppData\Local\Temp\ipykernel_13504\3674807392.py:6: DtypeWarning: Columns (14,15,21,22,23,24,25,26,31,33,49,50,55,56,66,69,70,71,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in files]
C:\Users\HIMANSHU SURYAVANSHI\AppData\Local\Temp\ipykernel_13504\3674807392.py:6: DtypeWarning: Columns (2,14,15,24,31,33,49,50,55,56,66,69,70,71,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in files]
C:\Users\HIMANSHU SURYAVANSHI\AppData\Local\Temp\ipykernel_13504\3674807392.py:6: DtypeWarning: Columns (14,15,23,24,25,26,31,33,49,50,55,56,66,69,70,71,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in files]


All monthly data has been successfully concatenated into 'final_concatenated_output.csv'.
